# **__O Problema__**

Você é um profissional encarregado de desenvolver um modelo preditivo de regressão para prever o valor dos custos médicos individuais cobrados pelo seguro de saúde.


A base de dados para este desafio será a seguinte:

https://github.com/feliperaro/fiap-pos-tech/blob/main/tech-challenge-1/resources/datasets/insurance.csv


Neste notebook, vamos explorar e preparar os dados, treinar diferentes modelos de machine learning e, finalmente, avaliar qual modelo oferece a melhor precisão para prever o valor dos custos médicos.


# Importação de Bibliotecas
Para iniciar, instalamos e importamos as bibliotecas necessárias para manipulação de dados, visualização e construção dos modelos de machine learning.


In [ ]:
# !conda install scikit-learn matplotlib numpy pandas seaborn -y
# !pip install scikit-learn matplotlib numpy pandas seaborn -y


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import seaborn as sns



# Carregamento de Dados
Download e leitura do dataset.



In [ ]:
def download_dataset():
    url = "https://raw.githubusercontent.com/feliperaro/fiap-pos-tech/refs/heads/main/tech-challenge-1/resources/datasets/insurance.csv"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

    with open("dataset.csv", "wb") as file:
        file.write(response.content)

download_dataset()
dataset = pd.read_csv("dataset.csv")



# Análise Exploratória dos Dados (EDA)
Nesta seção, exploramos as variáveis do dataset e analisamos suas distribuições.


In [ ]:
dataset.shape

In [ ]:
dataset.head()


In [ ]:
dataset.tail()

In [ ]:
dataset.info()


In [ ]:
dataset.nunique()

In [ ]:
dataset.duplicated().value_counts()

In [ ]:
dataset.drop_duplicates(inplace=True)

In [ ]:
string_columns = ["sex", "smoker", "region"]
for col in string_columns:
    print(f"{col}: {dataset[col].unique()}")


In [ ]:
data_to_translate = {
    "columns": {
        "age": "idade",
        "sex": "genero",
        "bmi": "imc",
        "children": "filhos",
        "smoker": "fumante",
        "region": "regiao",
        "charges": "custos"
    },
    "values": {
        "male": "masculino",
        "female": "feminino",
        "yes": "sim",
        "no": "nao",
        "northeast": "nordeste",
        "northwest": "noroeste",
        "southeast": "sudeste",
        "southwest": "sudoeste"
    }
}

for key, value in data_to_translate.items():
    if key == "columns":
        dataset = dataset.rename(columns=value)
    elif key == "values":
        dataset = dataset.replace(value)
...
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
dataset.hist(bins=50, figsize=(10, 10))

In [ ]:
dataset['grupo_idade'] = pd.cut(
    dataset['idade'],
    bins=[20, 35, 50, 65],  # Intervalos para as faixas
    labels=['Jovem', 'Meia-Idade', 'Sênior'],  # Rótulos para cada faixa
    right=False  # Inclui o limite inferior em cada faixa
)

age_cost_mean = dataset.groupby('grupo_idade', observed=True)['custos'].mean()

print(age_cost_mean)
age_cost_mean.plot(kind='bar', figsize=(10, 6), color='skyblue')

plt.title("Média do Custo por Faixa Etária")
plt.xlabel("Faixa Etária")
plt.ylabel("Média do Custo")
plt.show()


In [ ]:
colunas = ["genero", "fumante", "regiao", "idade", "imc"]

In [ ]:
plt.figure(figsize=(10, 6))
for coluna in colunas:
    sns.countplot(x=coluna, data=dataset, legend=False)
    plt.title('Distribuição')

    plt.xlabel(coluna)
    plt.ylabel("Quantidade")

    plt.show()


In [ ]:
for column in colunas:
    sns.boxplot(x=column, y='custos', data=dataset)
    plt.show()


# Preparação dos Dados


In [ ]:
dataset = pd.get_dummies(dataset, drop_first=True)
dataset.columns

In [ ]:
X = dataset.drop('custos', axis=1)
y = dataset['custos']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)
predicts = model.predict(X_test)


# Detecção e Remoção de Outliers

Outliers são valores que estão fora do padrão esperado e podem distorcer a análise e o desempenho dos modelos de machine learning. Nesta etapa, vamos identificar e remover esses valores para obter um conjunto de dados mais limpo e representativo usando o IQR (Intervalo Interquartil) para melhorar a qualidade dos dados.

1. **Calcular o IQR (Intervalo Interquartil)**:
   - Calculamos o primeiro quartil (Q1 == 25%) e o terceiro quartil (Q3 == 75%) de cada variável numérica. A diferença entre eles (Q3 - Q1) é o IQR, que representa a dispersão central dos dados.

2. **Definir Limites para Outliers**:
   - Usamos a regra comum para detectar outliers, considerando como "fora do padrão" os valores que estão abaixo de `25%` ou acima de `75%`.
   


**remove_outliers()**:
   - Cria uma máscara (filtro) para manter apenas as linhas com valores dentro dos limites normais, removendo linhas que possuem outliers em alguma variável numérica.

In [ ]:
def remove_outliers(df, features):
    mask = True
    for feature in features:
        mask &= (
            df[feature] >= (Q1[feature] - 1.5 * IQR[feature])) & (df[feature] <= (Q3[feature] + 1.5 * IQR[feature])
        )
    return df[mask]



In [ ]:
features_numericas = dataset.select_dtypes(include=['float64', 'int64']).columns
print("features_numericas:", features_numericas)

Q1 = dataset[features_numericas].quantile(0.25)
Q3 = dataset[features_numericas].quantile(0.75)

IQR = Q3 - Q1
print("Tamanho do DataFrame Original:", dataset.shape)

dataset_limpo = remove_outliers(dataset, features_numericas)
print("dataset_limpo.shape:", dataset_limpo.shape)


In [ ]:
X = dataset_limpo.drop('custos', axis=1)
y = dataset_limpo['custos']

X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

model = LinearRegression()
model.fit(X_train_cleaned, y_train_cleaned)
predicts_cleaned = model.predict(X_test_cleaned)


In [ ]:
mae_original = mean_absolute_error(y_test, predicts)
mse_original = mean_squared_error(y_test, predicts)

mae_cleaned = mean_absolute_error(y_test_cleaned, predicts_cleaned)
mse_cleaned = mean_squared_error(y_test_cleaned, predicts_cleaned)

print(f"Desempenho no dataset original - MAE: {mae_original}, MSE: {mse_original}")
print(f"Desempenho no dataset limpo - MAE: {mae_cleaned}, MSE: {mse_cleaned}")

In [ ]:
dataset = dataset_limpo

X = dataset.drop('custos', axis=1)
y = dataset['custos']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
}

results = {}
for model_name, model in models.items():
    cv_mse_scores = cross_val_score(
        model,
        X_train,
        y_train,
        cv=5,
        scoring='neg_mean_squared_error'
    )

    cv_mae_scores = cross_val_score(
        model,
        X_train,
        y_train,
        cv=5,
        scoring='neg_mean_absolute_error'
    )

    results[model_name] = {
        "Mean MSE (CV)": -cv_mse_scores.mean(),
        "MSE Scores (CV)": -cv_mse_scores,
        "Mean MAE (CV)": -cv_mae_scores.mean(),
        "MAE Scores (CV)": -cv_mae_scores
    }
    model.fit(X_train, y_train)

In [ ]:
for model_name, metrics in results.items():
    mse_scores = metrics['MSE Scores (CV)']
    mae_scores = metrics['MAE Scores (CV)']

    print(f"{model_name}:")
    print(f"Mean MSE (CV): {metrics['Mean MSE (CV)']:.2f}")
    print(f"MSE Scores (CV): {mse_scores}")
    print(f"Mean MAE (CV): {metrics['Mean MAE (CV)']:.2f}")
    print(f"MAE Scores (CV): {mae_scores}\n")


## Comparação de Modelos

Comparação dos três modelos – **Regressão Linear**, **Árvore de Decisão** e **Random Forest** – usando validação cruzada para avaliar o desempenho de cada um com as métricas **Mean Squared Error (MSE)** e **Mean Absolute Error (MAE)**.



In [ ]:
evaluation_results = {}
for model_name, model in models.items():
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    evaluation_results[model_name] = {
        "Mean Squared Error": mse,
        "R² Score": r2
    }

for model_name, metrics in evaluation_results.items():
    print(f"{model_name}: MSE = {metrics['Mean Squared Error']:.2f}, R² = {metrics['R² Score']:.2f}")

model_names = list(evaluation_results.keys())
mse_values = [metrics['Mean Squared Error'] for metrics in evaluation_results.values()]
r2_values = [metrics['R² Score'] for metrics in evaluation_results.values()]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(model_names, mse_values, color='skyblue')
plt.title('Mean Squared Error')
plt.xlabel('Models')
plt.ylabel('Mean Squared Error')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(model_names, r2_values, color='salmon')
plt.title('R² Score')
plt.xlabel('Models')
plt.ylabel('R² Score')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Conclusão

O **Random Forest** foi o modelo que apresentou o melhor desempenho geral, com o menor **Mean MSE** e **Mean MAE** entre os três modelos avaliados. Isso significa que o Random Forest:
- Possui menor variabilidade nas previsões, refletido pelo MSE mais baixo.
- Faz previsões mais próximas dos valores reais, indicado pelo menor MAE.

Portanto, com base nos resultados, o **Random Forest** é a melhor escolha para prever os custos médicos, oferecendo uma combinação equilibrada entre precisão e robustez para lidar com as variáveis deste dataset.

In [ ]:
best_model = RandomForestRegressor(random_state=42)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)
print(f"Best Model MSE: {mse:.4f}, R² Score: {r2:.4f}")


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valores Reais')
plt.ylabel('Valores Preditos')
plt.title('Valores Reais vs Preditos')
plt.show()


# Próximos Passos

Após a análise e comparação dos modelos, identificamos que o **Random Forest** apresentou o melhor desempenho geral na previsão dos custos médicos, tanto antes quanto depois da remoção dos outliers. A seguir, listamos algumas sugestões de próximos passos para aprimorar ainda mais o modelo e o processo de análise:

### 1. Testar Outros Modelos de Machine Learning
   - Experimentar modelos mais complexos, como **Gradient Boosting** ou **XGBoost**, que frequentemente têm um bom desempenho em problemas de regressão e podem capturar padrões mais sutis nos dados.
   - Avaliar o desempenho desses modelos comparado ao Random Forest para garantir que temos a melhor escolha.

### 2. Feature Engineering
   - Explorar novas combinações de variáveis (features), como **interações entre variáveis** ou **transformações logarítmicas**, que possam capturar melhor a relação entre variáveis e custos.
   - A criação de variáveis derivadas pode ajudar o modelo a entender padrões ocultos nos dados, especialmente se houver relações não lineares.

### 3. Implementação em Produção
   - Desenvolver um **pipeline de produção** para implementar o modelo, permitindo que ele faça previsões em tempo real.
   - Esse pipeline pode incluir a preparação dos dados (limpeza e transformação), aplicação do modelo treinado e armazenamento das previsões.
   - Monitorar o desempenho do modelo ao longo do tempo para ajustar caso haja mudanças nos dados ou nas condições do problema.

### 4. Monitoramento e Atualização do Modelo
   - Estabelecer um sistema de **monitoramento contínuo** para acompanhar o desempenho do modelo ao longo do tempo e detectar possíveis drifts de dados, que podem ocorrer se o perfil dos dados mudar.
   - Realizar re-treinamento periódico para atualizar o modelo com novos dados, garantindo que ele permaneça preciso e relevante.

Esses próximos passos ajudarão a garantir que o modelo seja robusto, confiável e eficiente em um cenário de produção, além de permitir um monitoramento e uma evolução contínua conforme o problema e os dados mudam.
